In [2]:
# Importing packages
import requests
import json
import time #will use this to pause execution for a few seconds

import numpy as np
import pandas as pd

from pandas import json_normalize

In [3]:
# Extracting Google Key
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

# extract key from a separate ini file where the key is stored
API_KEY = config['google']['api_key']

In [5]:
# The url format (backslash at the end is to continue to the next line, not part of the string)
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?\
location=19.7216%2C155.0849\
&radius=10000\
&type=restaurant\
&key=" + API_KEY

In [6]:
# The initial string for the api
endpoint_url_start = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [7]:
# Creating dataframes to be filled
hilo_common = pd.DataFrame(columns=['Name','place_id', 'Address','Coordinates',\
                                        'business_status', 'types'])
hilo_uncommon = pd.DataFrame(columns=['Name','place_id', 'Address','Coordinates',\
                                        'business_status', 'types'])
hilo_residential = pd.DataFrame(columns=['Name','place_id', 'Address','Coordinates',\
                                         'business_status', 'types'])

In [8]:
# list of places we want to look for
# (places tourists might visit and want to drop foodware off at)
collection_uncommon = ['art_gallery', 'campground', 'car_rental', 'museum',\
                       'night_club', 'spa', 'travel_agency']
collection_common = ['bus_station','light_rail_station', 'parking','taxi_stand',\
                     'train_station','transit_station']

collection_residential = ['apartment', 'condominium']

In [9]:
# storing Hilo-specific coordinates and search radius for uncommon places
central_cord = "19.68933, -155.08268"
central_radius = "9500"

In [10]:
# common/small radius searches info (same as distribution site searches)
cords = ["19.71890,-155.09343", "19.72473,-155.06600", "19.72600,-155.02977", "19.70562,-155.08258",\
         "19.70661,-155.05612", "19.69671,-155.11744", "19.67937,-155.07814", "19.67890,-155.03994",\
         "19.68336,-155.16312", "19.64372,-155.11469"]
radii = ["1500", "1500", "2500", "1500",\
         "1500", "2500", "2500", "3000",\
         "3000", "4000"]

In [11]:
def append_place(json_results, df):
    l = len(df)
    for i, val in enumerate(json_results['results']):
        df.loc[l + i] = [val.get('name', np.nan), 
                         val.get('place_id', np.nan),
                         val.get('vicinity', np.nan), 
                         val.get('geometry', np.nan).get('location', np.nan),
                         val.get('business_status', np.nan),
                         val.get('types', np.nan)]

In [13]:
# creating a dataset for the uncommon places
hilo_uncommon_full = pd.DataFrame(columns=['Name','place_id', 'Address','Coordinates',\
                                        'business_status', 'types'])
for i in range(5):
    for _type in collection_uncommon:
        endpoint_url = endpoint_url_start + "&location=" + central_cord + "&radius=" + \
        central_radius + "&region=us&type=" + _type + "&key=" + API_KEY
        
        params = {}
        
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_uncommon)
        
        
        time.sleep(1)
        
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token']
            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_uncommon)
            time.sleep(1)
    hilo_uncommon_full = pd.concat([hilo_uncommon_full, hilo_uncommon])

In [16]:
hilo_uncommon_full.drop_duplicates(subset=['place_id'], inplace=True)
hilo_uncommon_full.reset_index(drop=True, inplace=True)
hilo_uncommon_full['category'] = hilo_uncommon_full['types'].apply(lambda x: x[0])
hilo_uncommon_full.head()


,Name,place_id,Address,Coordinates,business_status,types,category
0,High Fire Hawaii Gallery & Studio,ChIJ_T9us6lMUnkRlCMwzLC2kgM,"223A Kilauea Avenue, Hilo","{'lat': 19.7217624, 'lng': -155.0848841}",OPERATIONAL,"[art_gallery, point_of_interest, establishment]",art_gallery
1,Bryan Booth Fine Furniture Antiques & Restoration,ChIJxe8BcldLUnkRiNArCZcT1rc,"296 Kilauea Avenue, Hilo","{'lat': 19.7209953, 'lng': -155.0843062}",OPERATIONAL,"[furniture_store, art_gallery, home_goods_stor...",furniture_store
2,Hawaii Arts & Crafts,ChIJZcKu36tMUnkRYDzCccwq9sA,Hilo,"{'lat': 19.7297111, 'lng': -155.0900232}",OPERATIONAL,"[art_gallery, point_of_interest, establishment]",art_gallery
3,The Exhibit Gallery Of Art And Events/ Hilo To...,ChIJmyq9L1ZLUnkRvxM1CLkv5TQ,"168 Keawe Street suite d, Hilo","{'lat': 19.72489089999999, 'lng': -155.087163}",OPERATIONAL,"[art_gallery, point_of_interest, establishment]",art_gallery
4,ONE GALLERY,ChIJFxPDp1dLUnkRi5d_DBepDzk,"186 Kamehameha Avenue, Hilo","{'lat': 19.72532709999999, 'lng': -155.0862412}",OPERATIONAL,"[art_gallery, point_of_interest, establishment]",art_gallery


In [18]:
# creating a dataset for the common places
for idx, circle in enumerate(cords):
    for _type in collection_common:
        endpoint_url = endpoint_url_start + "&location=" + circle + "&radius=" + \
        radii[idx] + "&region=us&type=" + _type + "&key=" + API_KEY

        params = {}

        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        append_place(results, hilo_common)

        time.sleep(1)
        
        # Pulling results from other pages
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token']
            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_common)
            time.sleep(1)

In [21]:
hilo_common.drop_duplicates(subset=['place_id'], inplace=True)
hilo_common.reset_index(drop=True, inplace=True)
hilo_common['category'] = hilo_common['types'].apply(lambda x: x[0])
hilo_common.head()

,Name,place_id,Address,Coordinates,business_status,types,category
0,Hele-On Mo'oheau Bus Terminal,ChIJLyDl7VdLUnkRIMdr1X1Hass,"329 Kamehameha Avenue, Hilo","{'lat': 19.7240487, 'lng': -155.0846213}",OPERATIONAL,"[bus_station, transit_station, point_of_intere...",bus_station
1,329 Kamehameha Ave Parking,ChIJgQ2O5ldLUnkR420lWey9j-Q,"329 Kamehameha Avenue, Hilo","{'lat': 19.72450809999999, 'lng': -155.0850912}",OPERATIONAL,"[parking, point_of_interest, establishment]",parking
2,301 Kamehameha Ave Parking,ChIJgQ2O5ldLUnkRvObF0Ps_Xd0,"301 Kamehameha Avenue, Hilo","{'lat': 19.72450809999999, 'lng': -155.0850912}",OPERATIONAL,"[parking, point_of_interest, establishment]",parking
3,Parking lot,ChIJ81DZMVZLUnkR0iqckIv2jzk,Hilo,"{'lat': 19.724665, 'lng': -155.087296}",NaN,"[premise, parking, point_of_interest, establis...",premise
4,Hilo Waianuenue Avenue @ Hilo High School,ChIJaSJmVau0U3kRDMuD498IXjc,United States,"{'lat': 19.721083, 'lng': -155.095889}",OPERATIONAL,"[transit_station, point_of_interest, establish...",transit_station


In [24]:
# creating a dataset for the residential places
hilo_residential_full = pd.DataFrame(columns=['Name','place_id', 'Address','Coordinates',\
                                         'business_status', 'types'])
for i in range(5):
    for idx, circle in enumerate(cords):
        for _type in collection_residential:
            endpoint_url = endpoint_url_start + "&location=" + circle + "&radius=" + \
            radii[idx] + "&region=us&keyword=" + _type + "&key=" +  API_KEY

            params = {}

            res = requests.get(endpoint_url, params = params)
            results =  json.loads(res.content)
            append_place(results, hilo_residential)

            time.sleep(1)
            
            # Pulling results from other pages
            while "next_page_token" in results:
                params['pagetoken'] = results['next_page_token']
                res = requests.get(endpoint_url, params = params)
                results =  json.loads(res.content)
                append_place(results, hilo_residential)
                time.sleep(1)
    hilo_residential_full = pd.concat([hilo_residential_full, hilo_residential])


In [27]:
# viewing dataset to see if it looks ok
hilo_residential_full.drop_duplicates(subset=['place_id'], inplace=True)
hilo_residential_full.reset_index(drop=True, inplace=True)
hilo_residential_full['category'] = hilo_residential_full['types'].apply(lambda x: x[0])
hilo_residential_full.head()

,Name,place_id,Address,Coordinates,business_status,types,category
0,ULULANI village Apartments,ChIJOQ1gC2JLUnkRQoNEVH0Alr8,"10 Ululani St, Hilo","{'lat': 19.7255751, 'lng': -155.0907569}",OPERATIONAL,"[point_of_interest, establishment]",point_of_interest
1,Riverside Apartments,ChIJY2nliKlMUnkR2dLh_bkmOqM,"333 Ohai St, Hilo","{'lat': 19.7277018, 'lng': -155.0898944}",OPERATIONAL,"[point_of_interest, establishment]",point_of_interest
2,Hilo Val-Hala Apartments,ChIJb4IPYalMUnkRekS9J-9psHE,"120 Puueo St, Hilo","{'lat': 19.7299063, 'lng': -155.0894607}",OPERATIONAL,"[real_estate_agency, point_of_interest, establ...",real_estate_agency
3,City View Apartments,ChIJERvpJDJLUnkRuRJCWO5EuJg,"230 Kapiolani St, Hilo","{'lat': 19.7184097, 'lng': -155.0879586}",OPERATIONAL,"[point_of_interest, establishment]",point_of_interest
4,Lincoln Courtside,ChIJzU7omfVLUnkRBnZvU4UxYSE,"400 Kinoole St, Hilo","{'lat': 19.7211984, 'lng': -155.0870105}",OPERATIONAL,"[point_of_interest, establishment]",point_of_interest


In [30]:
# exporting data sets to local csv files
hilo_common.to_csv('hilo_clct_common.csv')
hilo_uncommon_full.to_csv('hilo_clct_uncommon.csv')
hilo_residential_full.to_csv('hilo_clct_resid.csv')